In [180]:
#!usr/bin/env python3
#Author: Deepali L. Kundnani
#Institution: Georgia Institute of Technology (Georgia Tech)

Packages needed to be imported and input files need to be processed using

In [1]:
import pandas as pd
import argparse
import numpy as np

#parser = argparse.ArgumentParser(description='Getting TPM values from counts')
#parser.add_argument('--filename', default='exons_pc.counts', help="counts file, output from feature counts")
#parser.add_argument("-r", "--referencefasta" ,required=True, default='/storage/home/hcoda1/5/dkundnani3/p-fstorici3-0/rich_project_bio-storici/reference/sacCer2/sacCer2.fa', help="Specify the fasta file for reference of the sequenced Libraries. Make sure you have Ribosemap environment activated or have bedtools available to be used") 
#args= parser.parse_args()

#counts='GSE139242_CD4_counts.csv'
#counts='GSE175070_H9_counts.tsv'


Functions required to convert counts to tpm

In [17]:
import pandas as pd

def filterchr(df,col):
    """
    Filter out X and Y chromosomes

    Parameters:
    df (pd.DataFrame): A DataFrame containing necesarry details. Rows represent genomic regions, columns represent samples.
    col (num): Column index containing chromosome names

    Returns:
    pd.DataFrame: A DataFrame with required chromosomes
    """
    filtered_df = df[~df.iloc[:,col].str.contains('chrX|chrY|chrM')]

    return filtered_df

def convert_counts_to_tpm(df, gene_lengths):
    """
    Convert raw RNA-seq counts to TPM.

    Parameters:
    df (pd.DataFrame): A DataFrame containing RNA-seq counts. Rows represent genes, columns represent samples.
    gene_lengths (pd.Series): A Series containing the gene lengths in base pairs. The index should match the DataFrame index.

    Returns:
    pd.DataFrame: A DataFrame with TPM values.
    """
    # Step 0: Covert dataframes to float before any calculations
    df = df.astype(float) ; gene_lengths = gene_lengths.astype(float)
    
    # Step 1: Convert gene lengths from base pairs to kilobases
    gene_lengths_kb = gene_lengths / 1000
    
    # Step 2: Compute Reads Per Kilobase (RPK)
    rpk = df.div(gene_lengths_kb, axis=0)
    
    # Step 3: Compute scaling factor (sum of RPKs per sample)
    scaling_factors = rpk.sum(axis=0)
    
    # Step 4: Calculate TPM
    tpm = rpk.div(scaling_factors, axis=1) * 1e6
    
    return tpm


def convert_counts_to_rpkm(df, gene_lengths):
    """
    Convert raw RNA-seq counts to TPM.

    Parameters:
    df (pd.DataFrame): A DataFrame containing RNA-seq counts. Rows represent genes, columns represent samples.
    gene_lengths (pd.Series): A Series containing the gene lengths in base pairs. The index should match the DataFrame index.

    Returns:
    pd.DataFrame: A DataFrame with TPM values.
    """
    # Step 0: Covert dataframes to float before any calculations
    df = df.astype(float) ; gene_lengths = gene_lengths.astype(float)

    # Step 1: Compute scaling factor (sum of counts per sample)
    scaling_factors = df.sum(axis=0)
    
    # Step 3: Compute Reads Per Million (RPM)
    rpm = df.div(scaling_factors, axis=1) * 1e6
    
    # Step 4: Convert gene lengths from base pairs to kilobases
    gene_lengths_kb = gene_lengths / 1000

    # Step 5: Calculate RPKM
    rpkm = rpm.div(gene_lengths_kb, axis=0) 
    
    return rpkm


# Remove the top and bottom 5% of outliers from each column in tpm_df
def remove_outliers(df, lower_percentile=0.05, upper_percentile=0.95):
    """
    Removes outliers based on percentiles.

    Parameters:
    df (pd.DataFrame): Input DataFrame.
    lower_percentile (float): Lower percentile threshold (default: 0.05).
    upper_percentile (float): Upper percentile threshold (default: 0.95).

    Returns:
    pd.DataFrame: DataFrame with outliers removed.
    """
    filtered_df = df.apply(
        lambda x: x[(x >= x.quantile(lower_percentile)) & (x <= x.quantile(upper_percentile))],
        axis=0
    )
    return filtered_df

In [24]:
# Read counts file
counts_df = pd.read_csv('exonspc.counts', sep='\t',skiprows=1)
samples=counts_df.columns[6:counts_df.shape[1]].values
# Filtere Chromosome X, Y and M
counts_df = filterchr(counts_df,1)
min_count = 10  # Minimum count threshold
min_samples = len(samples)/2  # Minimum number of samples that must meet the threshold
# Convert counts to TPM
tpm_df = convert_counts_to_tpm(counts_df[samples], counts_df[['Length']].values)
# Merge TPM with Original file
merged_df = pd.concat([counts_df.iloc[:,0:6], tpm_df], axis=1)
# Save the TPM file for merging with other files
merged_df.to_csv('HEK_tpm.csv', index=False)

In [19]:
# Read counts file
counts_df = pd.read_csv('exonspc.counts', sep='\t', skiprows=1)
samples=counts_df.columns[6:counts_df.shape[1]].values
# Filter out X and Y chromosomes
counts_df = filterchr(counts_df, 0)
# Define filtering criteria
#min_count = 10  # Minimum count threshold
#min_samples = len(samples)/2  # Minimum number of samples that must meet the threshold
# Apply filtering
#counts_df = counts_df[(counts_df[samples] > min_count).sum(axis=1) >= min_samples]
# Convert counts to TPM & RPKM
tpm_df = convert_counts_to_tpm(counts_df[samples], counts_df[['Length']].values)
rpkm_df = convert_counts_to_rpkm(counts_df[samples], counts_df[['Length']].values)
#Converting the values to log2
#tpm_df = tpm_df.map(lambda x: 0 if x == 0 else np.log2(x))
#rpkm_df = rpkm_df.map(lambda x: 0 if x == 0 else np.log2(x))
# Save the TPM file for merging with other files
# Merge TPM with Original file
merged_df = pd.concat([counts_df.iloc[:,0:6], tpm_df], axis=1)
# Save the TPM file for merging with other files
merged_df.to_csv('HEK_tpm.csv', index=False)

In [20]:
counts_df = pd.read_csv('GSE139242_CD4T_protein.counts', sep='\t', header=0, index_col=0)
samples=counts_df.columns[1:counts_df.shape[1]].values
# Define filtering criteria
#min_count = 10  # Minimum count threshold
#min_samples = len(samples)/2  # Minimum number of samples that must meet the threshold
# Apply filtering
#counts_df = counts_df[(counts_df[samples] > min_count).sum(axis=1) >= min_samples]
# Convert counts to TPM & RPKM
tpm_df = convert_counts_to_tpm(counts_df[samples], counts_df[['Length']].values)
rpkm_df = convert_counts_to_rpkm(counts_df[samples], counts_df[['Length']].values)
#Converting the values to log2
#tpm_df = tpm_df.map(lambda x: 0 if x == 0 else np.log2(x))
#rpkm_df = rpkm_df.map(lambda x: 0 if x == 0 else np.log2(x))
# Save the TPM file for merging with other files
tpm_df.to_csv('CD4T_tpm.csv', index=True)
rpkm_df.to_csv('CD4T_rpkm.csv', index=True)

KeyError: "None of [Index(['Length'], dtype='object')] are in the [columns]"

In [12]:
# Apply the function to tpm_df
#tpm_df = remove_outliers(tpm_df)

from sklearn.preprocessing import MinMaxScaler
# Initialize the MinMaxScaler
scaler = MinMaxScaler()
# Apply min-max scaling to tpm_df
tpm_df_scaled = pd.DataFrame(scaler.fit_transform(tpm_df), columns=tpm_df.columns, index=tpm_df.index)


from scipy.stats import zscore
# Apply z-score normalization to tpm_df
tpm_df_zscore = tpm_df.apply(zscore, axis=0)


import numpy as np
# Apply decimal scaling to tpm_df
tpm_df_decimal_scaled = tpm_df.apply(lambda x: x / (10 ** np.ceil(np.log10(x.abs().max()))), axis=0)


from sklearn.preprocessing import RobustScaler
# Initialize the RobustScaler
scaler = RobustScaler()
# Apply robust scaling to tpm_df
tpm_df_robust_scaled = pd.DataFrame(scaler.fit_transform(tpm_df), columns=tpm_df.columns, index=tpm_df.index)


from sklearn.preprocessing import QuantileTransformer

# Initialize the QuantileTransformer
quantile_transformer = QuantileTransformer(output_distribution='uniform', random_state=0)

# Apply percentile transformation to tpm_df
tpm_df_percentile = pd.DataFrame(
    quantile_transformer.fit_transform(tpm_df),
    columns=tpm_df.columns,
    index=tpm_df.index
)



In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
# Define color mapping
colors = {
    'HEK293T_WT': '#D62728',
    'HEK293T_KO_T3_8': '#E377C2',
    'HEK293T_KO_T3_17': '#9467BD', 
    'H9': '#2CA02C',
    'CD4T': '#FF7F0E',
}

# Generate overlapping KDE plots for each column in tpm_df
plt.figure(figsize=(10, 8))
for column in tpm_df.columns:
    # Assign color based on sample type
    color = colors[column[:-2]]
    sns.kdeplot(tpm_df[column], label=column, color=color, fill=False, alpha=0.5)

plt.title('Overlapping KDE Plots for TPM Values')
plt.xlabel('TPM Values')
plt.ylabel('Density')
plt.legend(loc='upper right', fontsize='small', bbox_to_anchor=(1.25, 1))
plt.grid(axis='y', alpha=0.75)
plt.tight_layout()
plt.show()

# Generate overlapping KDE plots for each column in tpm_df
plt.figure(figsize=(10, 8))
for column in tpm_df_scaled.columns:
    # Assign color based on sample type
    color = colors[column[:-2]]
    sns.kdeplot(tpm_df_scaled[column], label=column, color=color, fill=False, alpha=0.5)

plt.title('Overlapping KDE Plots for TPM Values')
plt.xlabel('RPKM Values')
plt.ylabel('Density')
plt.legend(loc='upper right', fontsize='small', bbox_to_anchor=(1.25, 1))
plt.grid(axis='y', alpha=0.75)
plt.tight_layout()
plt.show()


KeyError: 'CD4_bloodinfant'

<Figure size 1000x800 with 0 Axes>

In [7]:
# Taking averages for each cell type

merged_df['HEK293T-KO-T3-17'] = merged_df.filter(like='HEK293T-KO-T3-17').mean(axis=1)
merged_df['HEK293T-KO-T3-8'] = merged_df.filter(like='HEK293T-KO-T3-8').mean(axis=1)
merged_df['HEK293T-WT'] = merged_df.filter(like='HEK293T-WT').mean(axis=1)

# Saving average TPM file
merged_df[['Geneid','HEK293T-WT','HEK293T-KO-T3-8','HEK293T-KO-T3-17']].to_csv('HEK_tpm_avg.csv', index=False)


In [8]:
# AGS counts analysis
counts_df = pd.read_csv('AGS_rnaseq_counts.tsv', sep='\t')
samples=counts_df.columns[0:7].values
tpm_df = convert_counts_to_tpm(counts_df[samples], counts_df[['Length']].values)
merged_df = pd.concat([counts_df.iloc[:,7:counts_df.shape[1]], tpm_df], axis=1)
merged_df.to_csv('AGS_tpm.csv', index=False)


In [74]:
# Correlation matrix of dataframe merged df
counts_df.iloc[:,6:counts_df.shape[1]].corr()


ValueError: could not convert string to float: 'A1BG'